In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers sentence-transformers googletrans==4.0.0-rc1
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install qwen-vl-utils[decord]==0.0.8
!pip install --upgrade --no-cache-dir transformers
!pip install --upgrade timm
!pip install --upgrade accelerate
!pip install --upgrade git+https://github.com/huggingface/transformers.git


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=cf5853b6cf2d178f209887689270c113e0cbed94a4691f7abfc682681a425433
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

In [4]:
# Import required libraries
import time
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from huggingface_hub import login
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [5]:
# Authenticate with Hugging Face (needed for gated models)
login()

# Download NLTK tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Define Student Email (from Assignment Instructions)
email_text = """Subject: Concerns About Professor X’s Conduct

Dear Dr. Ustaad,
I hope this email finds you well. I am writing to express my concerns about Professor X’s conduct 
during the Introduction to Zoology class last semester. On multiple occasions, Professor X made 
dismissive remarks about students’ questions and failed to provide clear feedback on assignments.  

Additionally, the grading seemed inconsistent and unfair, with no opportunity for clarification or 
appeal. 

I found this experience deeply frustrating and demotivating, and I believe it affected my performance 
in the course. I would appreciate it if the department could look into this matter and ensure that 
future students have a more positive and supportive learning environment. 

Thank you for your attention to this matter. 
Sincerely, 
Shaagird"""

In [7]:
# Load the Google T5 model once
t5_model = "google-t5/t5-3b"

In [8]:
import torch

# Automatically determine the best device
device = "cuda" if torch.cuda.is_available() and torch.cuda.mem_get_info(0)[0] > 1e9 else "cpu"

# Load summarization model
summarizer = pipeline("summarization", model=t5_model,device=-1)

# Load question answering model
qa_pipeline = pipeline("question-answering", model=t5_model,device_map="auto")

# For keyword extraction, use a separate model designed for sentence embeddings (e.g., 'all-MiniLM-L6-v2')
sbert_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)  # Consider using the best model for your purpose

# Initialize the Translator
translator = pipeline("translation_en_to_fr", model=t5_model,device_map="auto")  # English to French

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google-t5/t5-3b and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
translator_en_fr = pipeline("translation_en_to_fr", model=t5_model, device=-1)
translator_fr_en = pipeline("translation_fr_to_en", model=t5_model, device=-1)


Device set to use cpu
Device set to use cpu


In [9]:
def measure_time(task_function, *args, model_name="Unknown", **kwargs):
    start = time.time()
    result = task_function(*args, **kwargs) 
    end = time.time()
    elapsed_time = end - start
    print(f"{model_name} Execution Time: {elapsed_time:.2f} seconds")
    return result, elapsed_time

execution_times = {}

In [18]:
### **TASK 1: Summarization**
print("\nPerforming Summarization...")
summary_3b, execution_times["Summarization_3B"] = measure_time(
    summarizer, email_text, max_length=50, min_length=20, do_sample=False, model_name="Summarization (3B)"
)

# Extract the summarized text
summarized_text = summary_3b[0]['summary_text'] if isinstance(summary_3b, list) else summary_3b


Performing Summarization...
Summarization (3B) Execution Time: 35.86 seconds


In [19]:
### **TASK 2: Question Answering**
print("\nPerforming Question Answering...")
question = "What was the reason for the student's disappointment with Professor X?"
qa_3b, execution_times["QA_3B"] = measure_time(qa_pipeline, question=question, context=email_text, model_name="Question Answering (3B)")


Performing Question Answering...
Question Answering (3B) Execution Time: 0.47 seconds


In [20]:
### **TASK 3: Keyword Extraction (Based on Summarized Text)**
print("\nExtracting Keywords...")
start_time = time.time()

# Tokenize sentences from the summary
sentences = sent_tokenize(summarized_text)

# Encode the sentences using SentenceTransformer
embeddings = sbert_model.encode(sentences)

# Select top N most relevant sentences based on embedding norms
top_n = min(5, len(sentences))  # Ensure we don't extract more than available sentences
keyword_sentences = [sentences[i] for i in np.argsort(np.linalg.norm(embeddings, axis=1))[:top_n]]

# Record execution time
execution_times["Keyword_Extraction"] = time.time() - start_time

print(f"Keyword Extraction Execution Time: {execution_times['Keyword_Extraction']:.2f} seconds")
print(f"Extracted Keywords: {keyword_sentences}")


Extracting Keywords...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Keyword Extraction Execution Time: 0.03 seconds
Extracted Keywords: ['Concerns about Professor X’s conduct during the Introduction to Zoology class last semester .', 'professor X made dismissive remarks about students’ questions and failed to provide clear feedback on assignments .', 'grading seemed inconsistent and unfair,']


In [23]:
### **TASK 4: Paragraph translation**
print("\nTranslating to french...")
french_translation, execution_times["Translation_FR"] = measure_time(
    translator_en_fr, email_text, max_length=500, model_name="Translation to French"
)
print("\nTranslating back to english...")
back_translation, execution_times["Translation_EN"] = measure_time(
    translator_fr_en, french_translation[0]['translation_text'], max_length=500, model_name="Translation back to English"
)


Translation to French Execution Time: 79.95 seconds
Translation back to English Execution Time: 13.38 seconds


In [24]:
# Print Results
print("\n--- RESULTS ---")
print("\nSummarization:")
print(f"3B Model: {summary_3b[0]['summary_text']}")

print("\nQuestion Answering:")
print(f"3B Model: {qa_3b['answer']}")

print("\nKeyword Extraction:")
print(f"Extracted Keywords: {keyword_sentences[0]}")

print("\nTranslation:")
print(f"French Translation: {french_translation[0]['translation_text']}")
print(f"Back to English Translation: {back_translation[0]['translation_text']}")

# Print Execution Times for Each Task
print("\n--- Execution Times (Seconds) ---")
for task, time_taken in execution_times.items():
    print(f"{task}: {time_taken:.2f} sec")


--- RESULTS ---

Summarization:
3B Model: Concerns about Professor X’s conduct during the Introduction to Zoology class last semester . professor X made dismissive remarks about students’ questions and failed to provide clear feedback on assignments . grading seemed inconsistent and unfair,

Question Answering:
3B Model: questions and failed to provide clear feedback on

Keyword Extraction:
Extracted Keywords: Concerns about Professor X’s conduct during the Introduction to Zoology class last semester .

Translation:
French Translation: Objet : Préoccupations au sujet de la conduite du professeur X Cher Dr Ustaad, j’espère que ce courriel vous parvient en bonne santé. Je vous écris pour exprimer mes préoccupations au sujet de la conduite du professeur X au cours de la classe Introduction à la zoologie du semestre dernier.  plusieurs reprises, le professeur X a fait des remarques désobligeantes au sujet des questions des étudiants et
Back to English Translation: j’ai l’honneur de vous a